In [ ]:
import os
import shutil
import tempfile
import json
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import nibabel as nib
import scipy.ndimage as ndi
import pandas as pd
import random
import shutil
import statistics
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
from fastai.vision.all import *

from monai.losses import DiceCELoss,GeneralizedDiceFocalLoss,DiceLoss
from monai.inferers import sliding_window_inference
from monai.transforms import (
    AsDiscrete,
  
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    RandRotate90d,
    Transposed,
    ToTensord,
    Resized,
    RandLambdad,
    SelectItemsd,
    Lambdad,
    RandRotated,
    RandGaussianSmoothd,
    RandZoomd,
    RandGaussianNoised,
    RandScaleIntensityd

)

from monai.config import print_config
from monai.metrics import DiceMetric
from monai.networks.nets import SwinUNETR

from monai.data import (
    DataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
    SmartCacheDataset,
    IterableDataset,
    ThreadDataLoader,
    set_track_meta,
)


import torch
print_config()

In [ ]:
from monai.transforms.transform import MapTransform, RandomizableTransform
from typing import Any, Dict, Hashable, List, Mapping, Optional, Sequence, Tuple, Union
from monai.config import DtypeLike, KeysCollection, SequenceStr
class Slice_selector(RandomizableTransform,MapTransform):

    def __init__(
        self,
        keys: KeysCollection,
        prob: float = 1,
        #numslices: int = 8,
        #range: tuple = (1,6),
        allow_missing_keys: bool = False,
    ) -> None:
        MapTransform.__init__(self, keys, allow_missing_keys)
        RandomizableTransform.__init__(self, prob)
       

    def __call__(self, data: Mapping[Hashable, torch.Tensor],numslices=8,disrange=(1,6),randomize: bool = True) -> Dict[Hashable, torch.Tensor]:
        self.randomize(None)
        d = dict(data)

        for key in self.key_iterator(d):
          shape = d[key].size()
          slice_dis = random.randint(*disrange)
          span = slice_dis *numslices
          while span > shape[0]:
            slice_dis = random.randint(*disrange)
            span = slice_dis *numslices
          min_starting_slice = 0
          max_starting_slice = shape[0] - span
          starting_slice = random.randint(min_starting_slice, max_starting_slice)
          stack = []
          d[key] = d[key][starting_slice:starting_slice+numslices*slice_dis:slice_dis,:,:]
        return d


In [ ]:
train_FOLDER ='split_img\\train\\'
val_FOLDER ='split_img\\val\\'
test_FOLDER ='split_img\\test\\'
train = []
val = []
test = []

for element in os.listdir(train_FOLDER):
  train.append(element)
for element in os.listdir(val_FOLDER):
  val.append(element)
for element in os.listdir(test_FOLDER):
  test.append(element)

train = [{'image':train_FOLDER+x} for x in train]
test = [{'image':test_FOLDER+x} for x in test]
val = [{'image':val_FOLDER+x} for x in val]

In [ ]:
train_transforms = Compose(
    [
        LoadImaged(keys=["image"]),
        #Orientationd(keys=["image"], axcodes="RAS"),
        #Spacingd(
        #    keys=["image"],
        #    pixdim=(1.5, 1.5, 2.0),
        #    mode=("bilinear"),
        #),
        Resized(
            keys=["image"],
            spatial_size =(128,128,-1)
        ),
        #Resized(
        #    keys=["image"],
        #    spatial_size =(224,224,-1)
        #),
        RandZoomd(
            keys=["image"],
            prob=1,
            min_zoom=0.8,
            max_zoom=1.2,
        ),
        RandGaussianNoised(
            keys=["image"],
            prob=0.4,
            mean=0.0,
            std=0.04,
        ),
        RandFlipd(
            keys=["image"],
            spatial_axis=[0],
            prob=0.50,
        ),
        RandFlipd(
            keys=["image"],
            spatial_axis=[1],
            prob=0.50,
        ),
        RandRotate90d(
            keys=["image"],
            prob=0.50,
            max_k=3,
        ),
        RandRotated(
            keys=["image"],
            range_x = .1,
            prob=1,
        ),
        RandScaleIntensityd(
            keys=["image"],
            factors =0.20,
            prob=0.50,
        ),
        RandShiftIntensityd(
            keys=["image"],
            offsets=(0,.08),
            prob=0.50,
        ),
        Transposed(keys=["image"],indices=(3,0,1,2)),
        Slice_selector(keys=["image"]),
        ToTensord(keys=["image"]),
    ]
)

In [ ]:
val_transforms = Compose(
    [
        LoadImaged(keys=["image"], ensure_channel_first=True),
        Resized(
            keys=["image"],
            spatial_size =(128,128,-1)
        ),
        Transposed(keys=["image"],indices=(3,0,1,2)),
        ToTensord(keys=["image"]),
    ]
)

In [ ]:
train_ds = CacheDataset(data=train, transform=train_transforms, cache_rate=1.0,
                        #num_init_workers=4
                        )
train_loader = ThreadDataLoader(train_ds, num_workers=0, batch_size=16, shuffle=True)


Loading dataset:  11%|█         | 32/288 [01:25<12:55,  3.03s/it]

In [ ]:
val_ds = CacheDataset(data=val, transform=val_transforms, cache_rate=1.0,
                        #num_init_workers=4
                        )
val_loader = ThreadDataLoader(val_ds, num_workers=0, batch_size=1, shuffle=True)


In [ ]:
class SSBR_swin(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc7_res = nn.Linear(1000, 1)
        self.model = self.get_swin()
    
    def get_swin(self):
        swin_b = models.swin_t(pretrained=True)
        swin_b.features[0][0] = torch.nn.Conv2d(
            1, 96, kernel_size=(4, 4), stride=(4, 4))
        #swin_b = torch.nn.Sequential(*(list(swin_b.children())[:-1]))
        #swin_b.features[-1] = nn.Linear(768, 1)
        return swin_b
    def forward(self, x: torch.Tensor):
      x = F.relu(self.model(x.float()))
      x = x.view(-1, 1000)
      x = self.fc7_res(x)
      return x


class SSBR_resnet(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc7_res = nn.Linear(2048, 1)
        self.model = self.get_resnet()
    
    def get_resnet(self):
        resnet50 = models.resnet50(pretrained=True)
        resnet50.conv1 = torch.nn.Conv2d(
            1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        )
        resnet50 = torch.nn.Sequential(*(list(resnet50.children())[:-1]))
        return resnet50 
    def forward(self, x: torch.Tensor):
      x = F.relu(self.model(x.float()))
      x = x.view(-1, 2048)
      x = self.fc7_res(x)
      return x

class SSBR_vgg(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv6 = nn.Conv2d(
            512, 512, 1, stride=1, padding=0
        )  # in_channel, out_channel, kernel_size
        self.fc7 = nn.Linear(512, 1)
        self.model = self.get_vgg()
        self.avgpool =nn.AdaptiveAvgPool2d((1,1))
    def get_vgg(self):
        vgg16 = models.vgg16(pretrained=True)
        vgg16.features[0] = torch.nn.Conv2d(
            1, 64, kernel_size=3, stride=1, padding=1, bias=False
        )
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        vgg16.to(device)

        return vgg16.features
    def forward(self, x: torch.Tensor):
        x = self.model(x.float())
        x = F.relu(self.conv6(x))
        #x = self.avgpool(x)
        x = torch.mean(x, axis=(2, 3))
        x = x.view(-1, 512)
        x = self.fc7(x)
        return x

class SSBR_efficientnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc7_res = nn.Linear(1280, 1)
        self.model = self.get_efficientnet()
    
    def get_efficientnet(self):
        efficientnet = models.efficientnet_v2_s(pretrained=True)
        efficientnet.features[0][0] = torch.nn.Conv2d(
            1, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False
        )
        efficientnet = torch.nn.Sequential(*(list(efficientnet.children())[:-1]))
        return efficientnet
    def forward(self, x: torch.Tensor):
      x = F.relu(self.model(x.float()))
      x = x.view(-1, 1280)
      x = self.fc7_res(x)
      return x

class SSBR_convnext(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc7_res = nn.Linear(768, 1)
        self.model = self.get_convnext()
    
    def get_convnext(self):
        convnext = models.convnext_small(pretrained=True)
        convnext.features[0] = torch.nn.Conv2d(
            1, 96, kernel_size=(4, 4), stride=(4, 4))
        convnext = torch.nn.Sequential(*(list(convnext.children())[:-1]))
        return convnext
    def forward(self, x: torch.Tensor):
      x = F.relu(self.model(x.float()))
      x = x.view(-1, 768)
      x = self.fc7_res(x)
      return x

class SSBR_densenet(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc7_res = nn.Linear(1664, 1)
        self.model = self.get_densenet()
    
    def get_densenet(self):
        densenet169 = models.densenet169(pretrained=True)
        densenet169.features[0] = torch.nn.Conv2d(
            1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        )
        return densenet169.features
    def forward(self, x: torch.Tensor):
      x = F.relu(self.model(x.float()))
      x = torch.mean(x, axis=(2, 3))
      x = x.view(-1, 1664)
      x = self.fc7_res(x)
      return x

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TimeDistributed(SSBR_efficientnet()).to(device) 
SSBR_model = "swin"

In [ ]:
model

In [ ]:
def loss_order(scores_pred):
  scores_diff = scores_pred[:, 1:] - scores_pred[:, :-1]
  sigmoid = torch.sigmoid(scores_diff)
  loss = -torch.sum(torch.log(sigmoid))
  return loss
def loss_dist(scores_pred):
  scores_diff = scores_pred[:, 1:] - scores_pred[:, :-1]
  l1loss = torch.nn.SmoothL1Loss(reduction="sum")
  loss = l1loss(scores_diff[:, 1:], scores_diff[:, :-1])
  return loss
def loss_ssbr(scores_pred,alpha=1):
  loss = alpha*loss_order(scores_pred) + loss_dist(scores_pred)
  return loss

In [ ]:
epochs = 400
global_loss = float('inf')
model_path = "SSBR_experiments\\"+SSBR_model+".pth"
torch.backends.cudnn.benchmark = True
optimizer = torch.optim.RMSprop(model.parameters(), lr=1e-4)

In [ ]:
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n-------")
    train_loss = 0
    for batch_data in train_loader:
        model.train() 
        train_inputs = batch_data["image"].to(device)

        train_outputs = model(train_inputs)
        #print(train_outputs)
        t_loss = loss_ssbr(train_outputs)

        train_loss += t_loss
        t_loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    val_loss = 0
    model.eval()
    with torch.inference_mode():
        for val_batch_data in val_loader:
          v_inputs = val_batch_data["image"].to(device)
          v_outputs = model(v_inputs)
          #print(v_outputs)
          vloss = loss_ssbr(v_outputs)
          val_loss += vloss
        val_loss /= len(val_loader)
    if val_loss <global_loss:
        global_loss = val_loss
        torch.save(model.state_dict(), model_path)
        print("model was saved")
    else:
        print("model was not saved")

    train_loss /= len(train_loader)
    print(f"\nTrain loss: {train_loss:.5f} ")
    print(f"\nval loss: {val_loss:.5f} ")

In [ ]:
eval_transforms = Compose(
    [
        LoadImaged(keys=["image"], ensure_channel_first=True),
        Resized(
            keys=["image"],
            spatial_size =(128,128,-1)
        ),
        Transposed(keys=["image"],indices=(3,0,1,2)),
        ToTensord(keys=["image"]),
    ]
)

In [ ]:
train_FOLDER ='split_img\\train\\'
val_FOLDER ='split_img\\val\\'
test_FOLDER ='split_img\\test\\'
train = []
val = []
test = []

for element in os.listdir(train_FOLDER):
  train.append(element)
for element in os.listdir(val_FOLDER):
  val.append(element)
for element in os.listdir(test_FOLDER):
  test.append(element)

train = [{'image':train_FOLDER+x} for x in train]
test = [{'image':test_FOLDER+x} for x in test]
val = [{'image':val_FOLDER+x} for x in val]

In [ ]:
train_ds = CacheDataset(data=train, transform=eval_transforms, cache_rate=1.0,
                        #num_init_workers=4
                        )
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, 
                          #num_workers=4, 
                          pin_memory=True)

In [ ]:
test_ds = CacheDataset(data=test, transform=eval_transforms, cache_rate=1.0,
                        #num_init_workers=4
                        )
test_loader = DataLoader(test_ds, batch_size=1, shuffle=True, 
                          num_workers=4, 
                          pin_memory=True)

In [ ]:
val_ds = CacheDataset(data=val, transform=eval_transforms, cache_rate=1.0,
                        #num_init_workers=4
                        )
val_loader = DataLoader(val_ds, batch_size=1, shuffle=True, 
                          num_workers=4, 
                          pin_memory=True)

In [ ]:

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TimeDistributed(SSBR_vgg()).to(device) 
model_name = 'vgg'

In [ ]:
model_path = 'SSBRresults\\'+model_name+'.pth'
model.load_state_dict(torch.load(model_path))

In [ ]:
def output_json_score(data_loader,output):
  torch.load(model_path)
  model.eval()
  with torch.no_grad():
    for batch_data in tqdm(data_loader):
      img_path = batch_data["image_meta_dict"]["filename_or_obj"][0]
      img_name = os.path.split(img_path)[1]
      json_name = img_name.split(".")[0]+".json"
      if os.path.exists(output+json_name):
        continue
      inputs = batch_data["image"].to(device)
      outputs = model(inputs)
      out_file = {
          "img_name":img_name,
          "img_path":img_path,
          "scores":outputs.squeeze().tolist()
      }
      out_file = json.dumps(out_file, indent=4)
      with open(output+json_name, "w") as outfile:
        outfile.write(out_file)

In [ ]:
output_json_score(train_loader,'SSBRresults\\json\\'+model_name+'\\train\\')

In [ ]:
output_json_score(test_loader,'SSBRresults\\json\\'+model_name+'\\test\\')

In [ ]:
output_json_score(val_loader,'SSBRresults\\json\\'+model_name+'\\val\\')

In [ ]:
df = pd.read_excel("landmark.xlsx")
df=df.drop_duplicates(subset=['filename'])
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df

In [ ]:
summary

In [ ]:
score_df

In [ ]:
df

In [ ]:
#path =  "spaced_json/train/"
path = 'SSBR_experiments\\json\\efficientnet\\train\\'
score_df = pd.DataFrame( columns=["filename",
                            "femoral_start","femoral_end",
                            "bladder_start","bladder_end",
                            "liver_start","liver_end",
                            "lung_start","lung_end",
                            "heart_start","heart_end",
                            "stomach_start","stomach_end",
                            "esophagus_start","esophagus_end",
                            "spleen_start","spleen_end",
                            "pancreas_start","pancreas_end",
                            "duodenum_start","duodenum_end",
                            'small_intestine_start','small_intestine_end',
                            'large_intestine_start','large_intestine_end',
                            'rectum_start','rectum_end',
                            'adrenal_start','adrenal_end',
                            'gall_bladder_start','gall_bladder_end',
                            'kidney_start','kidney_end'])

for (dirpath, dirnames, filenames) in os.walk(path):
  for filename in filenames:
    name = filename.split(".")[0]
    f = open(dirpath+filename)
    data = json.load(f)
    scores=data["scores"]
    row= df[df['filename']==name]
    score_row = {}
    for c in score_df.columns[1:]:
      if row[c].item()!=-1:
        score_row[c]=scores[row[c].item()]
    score_row['filename'] = name  
    score_df = score_df.append(score_row,ignore_index=True)

summary = score_df.describe()
transform_min = summary['femoral_start']['mean']
transform_max = summary['esophagus_end']['mean']
def linear_transform(x, scale=1, min_value=0, max_value=1):
    x = x - min_value
    x = x * scale / (max_value - min_value)
    return x
def transform_scores(x):
  if (not np.isnan(transform_min)) & (not np.isnan(transform_max)):
      transformed = linear_transform(
          x,
          scale=100,
          min_value=transform_min,
          max_value=transform_max,
      )
      return transformed
      
from scipy.ndimage import gaussian_filter
def smooth_scores(x):
    smoothed = x.copy()
    not_nan = np.where(~np.isnan(x))
    not_nan_values = x[not_nan]

    # smooth scores
    smoothed[not_nan] = gaussian_filter(
        not_nan_values, sigma=10 / 1
    )

    return np.array(smoothed)

In [ ]:
score_df

In [ ]:
path = 'SSBRresults\\json\\vgg\\'
for (dirpath, dirnames, filenames) in os.walk(path):
    for filename in filenames:
        f = open(dirpath+"/"+filename)
        outpath = dirpath+"/"+filename
        data = json.load(f)
        scores=data["scores"]
        scores=np.array(scores)
        scores = smooth_scores(scores)
        scores = transform_scores(scores)
        data["post_scores"] = scores.tolist()

        out_file = json.dumps(data, indent=4)
        with open(outpath, "w") as outfile:
            outfile.write(out_file)

In [ ]:
df = pd.read_excel("landmark.xlsx")
df=df.drop_duplicates(subset=['filename'])
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df

In [ ]:
bad_labels ={'Pediatric-CT-SEG-0C78EBBE_img',
 'Pediatric-CT-SEG-0D4D6667_img',
 'Pediatric-CT-SEG-1D20EAD2_img',
 'Pediatric-CT-SEG-23DB510F_img',
 'Pediatric-CT-SEG-4A25C4F3_img',
 'Pediatric-CT-SEG-66065C99_img',
 'Pediatric-CT-SEG-AF83DAF1_img',
 'Pediatric-CT-SEG-C9494BA3_img',
 'Pediatric-CT-SEG-D1572D48_img',
 'Pediatric-CT-SEG-F50AD62F_img'}


In [ ]:
def gen_summary(model_name):
    post_score_df = pd.DataFrame( columns=["filename",
                              "femoral_start","femoral_end",
                              "bladder_start","bladder_end",
                              "liver_start","liver_end",
                              "lung_start","lung_end",
                              "heart_start","heart_end",
                              "stomach_start","stomach_end",
                              "esophagus_start","esophagus_end",
                              "spleen_start","spleen_end",
                              "pancreas_start","pancreas_end",
                              "duodenum_start","duodenum_end",
                              'small_intestine_start','small_intestine_end',
                              'large_intestine_start','large_intestine_end',
                              'rectum_start','rectum_end',
                              'adrenal_start','adrenal_end',
                              'gall_bladder_start','gall_bladder_end',
                              'kidney_start','kidney_end'])
    path =  "SSBRresults\\json\\"+model_name+"\\train\\"
    for (dirpath, dirnames, filenames) in os.walk(path):
        for filename in filenames:
            name = filename.split(".")[0]
            f = open(dirpath+filename)
            data = json.load(f)
            scores=data["post_scores"]
            row= df[df['filename']==name]
            score_row = {}
            for c in post_score_df.columns[1:]:
                if row[c].item()!=-1 and name not in bad_labels:
                    #print(filename,c,int(row[c].item()),len(scores))
                    score_row[c]=scores[int(row[c].item())]
            score_row['filename'] = name  
            post_score_df = post_score_df.append(score_row,ignore_index=True)
    summary = post_score_df.describe()
    return summary

In [ ]:
model_summary = gen_summary("efficientnet")

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = np.nanargmin(np.abs(array - value))
    return idx

In [ ]:
mae_score_df = pd.DataFrame( columns=["model","dataset",
                            "femoral_start","femoral_end",
                            "bladder_start","bladder_end",
                            "liver_start","liver_end",
                            "lung_start","lung_end",
                            "heart_start","heart_end",
                            "stomach_start","stomach_end",
                            "esophagus_start","esophagus_end",
                            "spleen_start","spleen_end",
                            "pancreas_start","pancreas_end",
                            "duodenum_start","duodenum_end",
                            'small_intestine_start','small_intestine_end',
                            'large_intestine_start','large_intestine_end',
                            'rectum_start','rectum_end',
                            'adrenal_start','adrenal_end',
                            'gall_bladder_start','gall_bladder_end',
                            'kidney_start','kidney_end','overall'])
std_score_df = pd.DataFrame( columns=["model","dataset",
                            "femoral_start","femoral_end",
                            "bladder_start","bladder_end",
                            "liver_start","liver_end",
                            "lung_start","lung_end",
                            "heart_start","heart_end",
                            "stomach_start","stomach_end",
                            "esophagus_start","esophagus_end",
                            "spleen_start","spleen_end",
                            "pancreas_start","pancreas_end",
                            "duodenum_start","duodenum_end",
                            'small_intestine_start','small_intestine_end',
                            'large_intestine_start','large_intestine_end',
                            'rectum_start','rectum_end',
                            'adrenal_start','adrenal_end',
                            'gall_bladder_start','gall_bladder_end',
                            'kidney_start','kidney_end','overall'])
models_result_path = "SSBRresults\\json\\"
landmark_df = pd.read_excel("landmark.xlsx")
for model in os.listdir(models_result_path):
  dataset_list = ["test","train","val"]
  if model=="totalseg":
      continue
  for dataset in dataset_list:
    result_path = models_result_path + model +"\\"+dataset+"\\"
    dic ={}
    
    model_summary = gen_summary(model)
    for (dirpath, dirnames, filenames) in os.walk(result_path):
      for filename in filenames:
        f = open(dirpath+filename)
        name = filename.split(".")[0]
        data = json.load(f)
        scores = data['post_scores']
        for landmark in mae_score_df.columns[2:-1]:
          start_mean = model_summary[landmark]["mean"]
          pred =find_nearest(scores,start_mean)
          real=landmark_df.loc[landmark_df['filename'] == name.split(".")[0]][landmark].values[0]
          if real==-1 or name in bad_labels:
            continue
          factor = 100/len(scores)
          scaled_pred = pred*factor
          scaled_real = real*factor
          mse_score = abs(scaled_pred-scaled_real)
          if landmark not in dic:
            dic[landmark] = []
          dic[landmark].append(mse_score)
          if mse_score>1000 and model == 'efficientnet':
            print(name,landmark,mse_score,scaled_pred,scaled_real,len(scores))
    mean_dic = {}
    std_dic = {}
    overall_error=[]
    for key,value in dic.items():
      #mean_dic[key] = sum(value)/len(value)
      mean_dic[key] = statistics.mean(value)
      std_dic[key] =statistics.stdev(value)
      overall_error = overall_error+value
      
    mean_dic['overall'] = statistics.mean(overall_error)
    mean_dic["model"] = model
    mean_dic["dataset"] = dataset
    
    std_dic['overall'] = statistics.stdev(overall_error)
    std_dic["model"] = model
    std_dic["dataset"] = dataset
    mae_score_df = mae_score_df.append(mean_dic,ignore_index=True)
    std_score_df = std_score_df.append(std_dic,ignore_index=True)

In [ ]:
mae_score_df

In [ ]:
mae_score_df.to_csv("mae.csv")

In [ ]:
rmse_score_df = pd.DataFrame( columns=["model","dataset",
                            "femoral_start","femoral_end",
                            "bladder_start","bladder_end",
                            "liver_start","liver_end",
                            "lung_start","lung_end",
                            "heart_start","heart_end",
                            "stomach_start","stomach_end",
                            "esophagus_start","esophagus_end",
                            "spleen_start","spleen_end",
                            "pancreas_start","pancreas_end",
                            "duodenum_start","duodenum_end",
                            'small_intestine_start','small_intestine_end',
                            'large_intestine_start','large_intestine_end',
                            'rectum_start','rectum_end',
                            'adrenal_start','adrenal_end',
                            'gall_bladder_start','gall_bladder_end',
                            'kidney_start','kidney_end','overall'])
rstd_score_df = pd.DataFrame( columns=["model","dataset",
                            "femoral_start","femoral_end",
                            "bladder_start","bladder_end",
                            "liver_start","liver_end",
                            "lung_start","lung_end",
                            "heart_start","heart_end",
                            "stomach_start","stomach_end",
                            "esophagus_start","esophagus_end",
                            "spleen_start","spleen_end",
                            "pancreas_start","pancreas_end",
                            "duodenum_start","duodenum_end",
                            'small_intestine_start','small_intestine_end',
                            'large_intestine_start','large_intestine_end',
                            'rectum_start','rectum_end',
                            'adrenal_start','adrenal_end',
                            'gall_bladder_start','gall_bladder_end',
                            'kidney_start','kidney_end','overall'])
models_result_path = "SSBRresults\\json\\"
landmark_df = pd.read_excel("landmark.xlsx")
for model in os.listdir(models_result_path):
  if model=="totalseg":
      continue
  dataset_list = ["test","train","val"]
  for dataset in dataset_list:
    result_path = models_result_path + model +"\\"+dataset+"\\"
    dic ={}
    model_summary = gen_summary(model)
    for (dirpath, dirnames, filenames) in os.walk(result_path):
      for filename in filenames:
        f = open(dirpath+filename)
        name = filename.split(".")[0]
        data = json.load(f)
        scores = data['post_scores']
        for landmark in mse_score_df.columns[2:-1]:
          start_mean = model_summary[landmark]["mean"]
          pred =find_nearest(scores,start_mean)
          real=landmark_df.loc[landmark_df['filename'] == name.split(".")[0]][landmark].values[0]
          if real==-1 or name in bad_labels:
            continue
          factor = 100/len(scores)
          scaled_pred = pred*factor
          scaled_real = real*factor
          mse_score = ((scaled_pred-scaled_real)**2)
          if landmark not in dic:
            dic[landmark] = []
          dic[landmark].append(mse_score)
          if mse_score>1000 and model == 'efficientnet':
            print(name,landmark,mse_score,scaled_pred,scaled_real,len(scores))
    mean_dic = {}
    std_dic = {}
    overall_error=[]
    for key,value in dic.items():
      #mean_dic[key] = sum(value)/len(value)
      mean_dic[key] = statistics.mean(value)**(1/2)
      std_dic[key] =statistics.stdev(value)**(1/2)
      overall_error = overall_error+value
      
    mean_dic['overall'] = statistics.mean(overall_error)**(1/2)
    mean_dic["model"] = model
    mean_dic["dataset"] = dataset
    
    std_dic['overall'] = statistics.stdev(overall_error)**(1/2)
    std_dic["model"] = model
    std_dic["dataset"] = dataset
    rmse_score_df = rmse_score_df.append(mean_dic,ignore_index=True)
    rstd_score_df = rstd_score_df.append(std_dic,ignore_index=True)